# 맵리듀스로 그리는 빅데이터 지도

![img](http://www.hostdime.in/blog/wp-content/uploads/2015/09/mapreduce-logo.jpg)

**context**
1. [빅데이터 처리의 기본 패턴](#빅데이터-처리의-기본-패턴)
2. [Split Apply Combine](#split-apply-combine)
3. [맵리듀스의 원리](#맵리듀스의-원리)
4. [분산환경의 빅데이터 처리](#분산환경의-빅데이터-처리)
5. [하둡과 스파크](#하둡과-스파크)
6. [함수, 파이썬의 1급 시민](#함수-파이썬의-1급-시민)
7. [함수형 프로그래밍과 맵리듀스](#함수형-프로그래밍과-맵리듀스)
8. [맵리듀스 예제](#맵리듀스-예제)


## 빅데이터 처리의 기본 패턴

본격적으로 시작하기에 앞서, 빅데이터를 처리하기 위한 방법을 한번 생각해보세요! 번뜩 떠오르시나요? 그렇다면 스슥 넘어가셔도 될 것 같습니다. :)

빅데이터를 처리하기 위해선.. 말 그대로 데이터 용량도 크고 처리시간이 많이 소요될 것입니다. 성능이 좋은 컴퓨터를 써야겠지요(스케일업). 혹은 컴퓨터를 여러대로 연결한 뒤 각 컴퓨터에서 분산처리를 하도록 해도 될 것 같습니다. (스케일아웃)

빅데이터라는 용어는 시장에서 다소 버즈 워드(buzz word) 취급을 받는 경향도 있습니다. 빅데이터에 내재된 활용하여 긴밀하게 전략을 구사하지 않으면 21세기의 경쟁 사회에서 뒤쳐질 수도 있다는 주장에 딱히 반대를 하지는 않지만, 과연 빅데이터란 무엇이고 어떻게 처리되는 것인지 정확한 메커니즘은 다소 혼란스럽습니다. 빅데이터를 다루는 솔루션이라는 하둡, 스파크 등의 생소한 이름들도 빅데이터 처리 메커니즘이란 어떤 것인지 한꺼풀 들쳐보기 다소 부담스럽게 합니다.

오늘 우리가 다룰 **맵리듀스(Mapreduce)**라는 것은 하둡, 스파크 등 빅데이터를 다루는 솔루션들의 가장 근간이 되는 프로그래밍 모델입니다. 약간은 추상적인 개념이라 바로 직관적으로 이해하기 어려워하기도 합니다만, 괜찮습니다. 내가 처리해야 할 무언가 양이 매우 많다? 어떻게 하시겠습니까? 실은 뻔합니다. **1)많은 것을 잘게 나누어서 2)각각을 처리한 후에 3)처리된 것들을 모아서 통합 결과물을 낸다** 정도의 생각을 하시겠죠? 그게 바로 맵리듀스 개념의 전부입니다. 실은 아주 상식적인 개념이고, 우리도 일상에서 아주 많이 쓰고 있는 개념이기 때문에 적응하는데 어렵지 않으실 것입니다.

추상적인 데이터를 얘기하면 혼란스러우니까, 아주 일상적인 비유로 맵리듀스 개념을 설명해 보고자 합니다.

> 어느 만석꾼 부자집 이야기

여러분이 조선시대 어느 만석꾼 부자집 일꾼이라고 합시다. 하루는 주인 양반이 이런 지시를 내립니다. "이보게, 이 쌀 한그릇에 모래가 들어갔네. 모래를 다 골라내게!"

어떻게 하시겠습니까? 한그릇 분량의 쌀은 대충 2000 알갱이 정도 된다고 합니다.참고 여러분은 매초 한알씩 쉬지않고 열심히 쌀알갱이와 모래를 골라냈습니다. 그래서 30여분이 지나서 '주인님 다 되었습니다!' 하고 보고할 수 있게 되었습니다.

그랬더니, 주인 양반 왈, "아이구 참 잘했네. 그런데 보니 이 쌀을 퍼낸 저 가마니 안에 온통 이렇게 모래가 들어가 있지 뭔가, 이보게, 이거 전부다 좀 골라내 주게. 이거 다 골라내지 않으면 오늘 밥은 없네." <br/>
큰일났습니다. 쌀 한가마니는 대략 4000그릇이라고 합니다. 아까 방법으로 모래를 골라냈다가는… 대략 2400시간, 딱 100일 동안 밥을 못먹겠군요.

이 소식을 들은 동네의 친구 100명이 몰려들었습니다. 이 착하고 성실한 친구들은 하루종일 쉬지도 않고 열심히 여러분과 같은 방법으로 모래를 골라서 깨끗한 쌀알만을 추려 모아 다시 깨끗한 쌀 한가마니를 만들어 냈습니다. 이렇게 맵리듀스 마을의 친구들은 멋진 분산처리를 통해 여러분을 위기에서 구해낼 수 있었습니다. 그들이 쓴 방법은 바로,

- 1) 한가마니의 쌀을 100등분으로 잘게 나누어서
- 2) 한명당 그 1/100을 가져가 각각 처리(한알씩 골라내기)를 열심히 한 후에
- 3) 각자가 분리해 낸 깨끗한 쌀을 한데 모아 다시 한가마니를 만들어 냈습니다.

바로 정확히 맵리듀스의 개념과 일치하는군요.

> 빅데이터 처리의 기본 패턴

눈치채셨겠지만, 일꾼 혼자서는 처리할 수 없는 일거리를 여럿이 나눠서 처리하는 것과, 컴퓨터 한대로는 처리할 수 없는 분량의 데이터를 여러 대가 나눠서 처리하는 작업의 원리는 같습니다. 이것을 데이터분석의 **Split-Apply-Combine Strategy**라고 합니다. Mapreduce는 이런 모델의 한 특수한 형태라고 할 수 있습니다.[참고, 위키피디아](https://en.wikipedia.org/wiki/MapReduce) 다음 스텝에서 이 전략에 대해 좀더 살펴봅시다.

## Split Apply Combine

아래는 `Split-Apply-Combine Strategy`를 명쾌하게 설명해주고 있는 글입니다. 한번 읽어보고 가실까요?

- [Split-Apply-Combine Strategy for Data Mining](https://medium.com/analytics-vidhya/split-apply-combine-strategy-for-data-mining-4fd6e2a0cc99)

위 글에 따르면 1)많은 것을 잘게 나누어서(**Split**) 2)각각을 처리한 후에(**Apply**) 3)처리된 것들을 모아서 통합 결과물을 낸다(**Combine**) 는 전략은 SQL의 GroupBy 문, R의 plyr 함수 등 다양한 곳에 적용되고 있습니다. 오늘 우리가 살펴볼 Mapreduce 도 그런 좋은 사례가 될 것입니다.

그럼 Mapreduce에 대해 위의 관점으로 좀더 구체적으로 들여다볼까요? 아래 링크는 2004년 구글의 제프리 딘(Jeffrey Dean)과 산제이 기마와트(Sanjay Ghemawat)가 발표한 MapReduce: Simplified Data Processing on Large Clusters 란 논문입니다. 이 논문이 오늘날 빅데이터용 클러스터 컴퓨팅에 사용되는 주요 프로그래밍 모델인 맵리듀스를 최초로 제안한 공식 자료입니다.

- [MapReduce: Simplified Data Processing on Large Clusters](https://static.googleusercontent.com/media/research.google.com/ko//archive/mapreduce-osdi04.pdf)

논문에서 정리한 Mapreduce의 엄밀한 정의는 다음 스텝에서 다루겠습니다. 아래 그림은 이 논문의 Mapreduce 개념도를 바탕으로 이전 스텝의 일꾼 비유를 Split-Apply-Combine Strategy 관점으로 재구성한 것입니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/F-35v2-1.max-800x600.png)

Mapreduce는 크게 map() 함수와 reduce() 함수로 구성되어 있습니다. 그래서 합쳐서 Mapreduce 입니다.

- `map()` 함수는 Split된 부분 데이터를 가져다가 어떤 특별한 조작을 가하는, **Apply** 역할을 하는 함수입니다. 일꾼 비유에서 본다면, 모래섞인 쌀을 가져다가 한알 한알 쌀과 모래로 분리하고 있는 친구가 이 `map()` 함수 역할을 하고 있습니다.
- `reduce()` 함수는 `map()` 함수가 만들어낸 결과물을 어떤 기준에 따라 한군데로 다시 모아내는, **Combine** 역할을 하는 함수입니다. 일꾼 비유에서 본다면, 친구들이 분리해 놓은 쌀과 모래들을 가져다가 쌀만 한 가마니에 모아담는 일꾼이 `reduce()` 함수 역할을 하고 있습니다.

그럼 다음 논문에서는 Mapreduce의 저자들의 생각을 좀더 디테일하게 살펴 봅시다.

## 맵리듀스의 원리

맵리듀스는 하나의 컴퓨터에서 하던 작업을 여러 개의 컴퓨터에서 처리하도록 분산시키는 프로그래밍 모델입니다. 이 모델은 처리하고자 하는 작업을 여러 컴퓨터에 자동으로 분할하고 지역 처리 및 클러스터 노드 간 통신을 하기 위해 사용합니다.

### 맵리듀스 논문

2004년 구글의 제프리 딘(Jeffrey Dean)과 산제이 기마와트(Sanjay Ghemawat)는 MapReduce: Simplified Data Processing on Large Clusters 란 논문을 발표했습니다.

이 논문이 오늘날 빅데이터용 클러스터 컴퓨팅에 사용되는 주요 프로그래밍 모델인 맵리듀스를 최초로 제안한 공식 자료입니다. 아래 링크에 가시면 논문의 원문을 확인할 수 있습니다. 처음 접하는 사람에게는 조금 어려운 내용일 수도 있습니다. 읽기 힘드신 분은 논문의 `Abstract`, `2.1 Types`, `Figure 1: Execution overview`, `8. Conclusions` 단락만 읽으셔도 됩니다.

- [MapReduce: Simplified Data Processing on Large Clusters](https://static.googleusercontent.com/media/research.google.com/ko//archive/mapreduce-osdi04.pdf)

그리고 추가로 도움이 될만한 좋은 자료가 있습니다. 논문의 저자들이 작성한 요약 설명 슬라이드 자료입니다.

- [MapReduce:
Simplified Data Processing on Large Clusters](https://research.google.com/archive/mapreduce-osdi04-slides/index.html)

아래 그림은 이 슬라이드의 3페이지에 언급된 Mapreduce의 프로그래밍 모델 개요입니다.

![Img](https://aiffelstaticprd.blob.core.windows.net/media/images/F-35v2-2.max-800x600.png)

Mapreduce는 `map`와 `reduce` 2개의 함수만 강조되고 있는것 같지만, 이것 또한 실은 Split 단계를 전제로 하고 있습니다. `map` 함수의 인자로 `in_key`를 받는 것을 보면 알 수 있습니다.

위 그림에서 `map` 함수를 보면 `in_key`와 `in_value`를 인자로 갖습니다. `in_key`란 바로 Split의 결과고 생긴 partitioning 키값입니다. 일꾼 예제에서라면 0번~99번 친구의 키값이라고 할 수 있겠습니다. 이 키값은 최종 Output에는 반영되지 않습니다. 어떤 쌀알을 구분한 사람이 몇번 친구인지는 중요하지 않으니까요.

`map`함수는 Split된 데이터를 가져다가 `out_key`와 `intermediate_value`의 리스트로 변환합니다. out_key는 map함수가 결과물을 구분하는 기준 키값입니다. 일꾼 예제라면 0:쌀, 1:모래 같은 형태가 되겠죠. 그래서 일꾼 예제의 map 친구들은 길이 2인 list에 각각 깨끗한 쌀과 모래를 담아서 리턴할 것입니다.

`reduce` 함수의 입력은 여러 `map`함수의 `intermediate_value`들을 out_key별로 구분해서 리스트업한 것을 입력으로 합니다. 일꾼 예제라면 100명의 친구들이 모아 준 쌀 100덩어리와 모래 100덩어리가 입력이 되겠군요. 이것을 가지고 `recuce` 함수는 `out_key` 기준으로 sum을 할 것입니다. 그래서 `reduce` 함수인 일꾼은 최종적으로 `out_value`의 list를 출력합니다. 즉, 전체 쌀과 전체 모래를 모아놓은 길이 2의 list가 최종 출력이 될 것입니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/F-35v2-3.max-800x600.png)

위 그림은 Mapreduce의 실행(Execution) 모델입니다. M이 Input을 입력받아 Intermediate로, R이 Intermediate를 `Out_key`기준으로 Grouping한 것을 입력으로 받아 Output을 출력하는 구조로 진행되는 것을 볼 수 있습니다.

여기서 초기 Mapreduce의 한가지 약점이라고도 할 수 있는 특징을 알 수 있습니다. `reduce` 함수는 모든 `map` 함수가 중간 결과물을 리턴할 때까지 기다렸다가 한꺼번에 그룹핑한 결과가 나올때까지 기다립니다. 일꾼 예제를 예로 든다면, 어느 한 친구가 작업이 많이 늦어져서 쌀과 모래가 전부 취합되지 않는다면 우리의 일꾼은 마지막 한 친구의 작업이 끝날때까지 한없이 기다려야 한다는 것입니다. (뭔가 다른 일을 할수도 있었을텐데요.) 이런 시스템이 가지고 있는 잠재적 약점이 있습니다.

이 Execution model 그림에서 보면 `map` 함수 뿐 아니라 `reduce` 함수도 병렬수행 가능하다는 것을 알 수 있습니다. 우리 일꾼 예제에서 `reduce` 함수 역할을 일꾼 혼자서 하는 걸로 하였지만 실제로는 `reduce` 함수도 여러 개를 병렬수행 가능합니다.

## 분산환경의 빅데이터 처리

맵리듀스에 대한 대략적인 감이 잡히셨을 것 같습니다. 아래 위키피디아 사이트에서 다시 한번 맵리듀스에 대해서 읽어봅시다!

- [Wikipedia: MapReduce](https://en.wikipedia.org/wiki/MapReduce)

다소 이해하기 어려운 부분들이 많았을 겁니다. 아래 그림을 다시 한번 확인해보세요. 

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_evDkPe7.max-800x600.png)

우선 **맵리듀스의 정의**와 관련된 개념이 나옵니다.

- 클러스터 위에 동작하는 알고리즘
- 프로그래밍 모델

그리고 **맵리듀스의 로직**을 소개합니다.

- Map, Reduce, Shuffle

또.. **함수형 프로그래밍(Functional Programming)**이 눈에 띄네요. 함수형 프로그래밍도 중요한 개념입니다.

위 지도를 따라서 맵리듀스를 지탱하는 기술에는 어떤 개념들이 있을지 하나씩 따라가봅시다.

### 병렬, 분산, 클러스터 컴퓨팅

위에서 우리는 컴퓨터 한대로 감당할 수 없는 빅데이터를 처리하기 위해서는 병렬 컴퓨팅이나 분산 컴퓨팅이 필요하다는 언급을 했습니다. 이 두가지 개념을 혼용해서 사용하는 경우가 많지만, 엄밀하게는 다른 개념이긴 합니다. 한번 살펴봅시다!


**병렬 컴퓨팅(Parallel Computing)**

이 개념은 <u>한대의 컴퓨터 안에서 CPU 코어를 여러개 사용</u>하여 한대의 컴퓨터가 처리하는 데이터의 총량과 처리속도를 증가시키자는 것입니다. 관련된 개념으로는 아래와 같습니다.

- 멀티 프로세스 (Multiprocessing)
    - *프로세스*는 컴퓨터에서 연속적으로 실행되고 있는 컴퓨터 프로그램을 의미합니다. 스케줄링의 대상이 되는 작업(task)라는 용어로도 쓰입니다. 프로세스 관리는 OS가 합니다.
    - 멀티 프로세스는 2개 이상의 프로세스를 사용하는 것을 말합니다.
- 멀티 쓰레드 (Multithreading)
    - *스레드(thread)*는 프로세스 내에서 실행되는 흐름의 단위를 의미합니다.
    - 일반적으로 한 프로그램은 한 스레드를 가지나, 멀티 쓰레드는 2개 이상의 쓰레드를 동시에 사용하고 있는 것을 말합니다.
    
    
**분산 컴퓨팅**

<u>여러 대의 컴퓨터가 네트워크로 연결된 상황을 전제</u>로 하는 것입니다. 따라서 아래와 같은 네트워크 용어들이 중요하게 다뤄집니다.

- P2P (Peer to Peer)
- HTTP
- Network

**클러스터 컴퓨팅**

위 그림을 보면 클러스터 컴퓨팅이 위의 병렬 컴퓨팅과 분산 컴퓨팅을 통합하는 개념처럼 제시되고 있습니다. 실제로 이 용어는 분산 컴퓨팅과 매우 혼용되고 있습니다.

- **여러대의 컴퓨터들이 연결되어 하나의 시스템처럼 동작**하는 컴퓨터들의 집합
- 보통 **노드**와 **관리자**로 구성되어 있습니다.
    - 클러스터 노드 : 프로세싱 자원을 제공하는 시스템
    - 클러스터 관리자 : 노드를 서로 연결하여 단일 시스템처럼 보이게 만드는 로직을 제공
- 대표적인 기술
    - 병렬 컴퓨팅(Parellel Computing)
    - 분산 컴퓨팅(Distributed Computing)
    - 클라우드 텀퓨팅(Cloud Computing)


클러스터는 여러대의 컴퓨터들이 연결되어 있는 것입니다. 이때 컴퓨터를 어떻게 연결하고 컴퓨팅 자언을 관리할 것인지에 따라서 많은 클러스터 관련 기술이 알려져 있습니다. 밑에서 소개할 하둡, 스파크와 같은 시스템들은 이런 클러스터 컴퓨팅을 전제로 설계된 시스템입니다.



## 하둡과 스파크

<br/>

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/hadoop.max-800x600.jpg)

하둡은 대용량 데이터를 분산 처리할 수 있는 자바 기반의 오픈소스 프레임워크 입니다. 2004년에 구글에서 Mapreduce가 발표되고, 여기에 GFS(Google File System)이라는 구글의 분산 파일 시스템이 나오면서 이미 천문학적인 크기가 된 인터넷 검색 데이터를 분산처리하는 기술을 통해 구글이 기술적 우위를 공고히 하게 됩니다. 특히 GFS는 구글에서 공개를 하지 않았습니다. 이에 아파치 재단(Apache Foundation)에서는 구글의 맵리듀스 및 GFS에 해당하는 HDFS(Hadoop File system)를 포함하는 하둡(Hadoop)이라는 오픈소스 프로젝트를 2006년 4월에 발표하게 됩니다. 이후 하둡은 오픈소스라는 강점을 내세워 대략의 데이터 처리를 지원하는 사실상의 표준 프레임워크로 군림하게 됩니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/spark.max-800x600.png)

그러나, 하둡이 가지는 약점이 있었습니다. 이것은 문제는 Mapreduce의 문제이기도 합니다. map함수가 모두 종료해야 reduce함수가 실행된다는 가져오는 성능 손실이 컸기 때문입니다. 이것이 사실상 하둡으로 실시간 서비스를 제공하는 것을 불가능하게 했고, 하둡은 빅데이터 기반의 배치성 통계작업에만 주로 투입되고 있습니다.
반면 2009년에 처음 발표된 스파크(Spark)는 종래의 Mapreduce 기능에서 Map함수가 전부 종료되지 않았더라도 Map의 결과를 스트리밍하는 방식으로, Map의 결과가 다 나와야만 Reduce를 수행한다는 전제를 깨버리게 됩니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_13_x1MFO6H.max-800x600.png)

맵리듀스 논문의 그림을 다시한번 가져와 보겠습니다. 그림을 보면 한가지 설계적 문제가 눈에 띕니다. 그것은 바로 map 함수의 결과가 디스크에 저장되고, 그걸 reduce 함수가 다시 읽어와야 한다는 점입니다. 이것이 주는 성능상의 손실이 큽니다. 반면 스파크는 인메모리 데이터 엔진을 통해 초기 맵리듀스의 성능상 한계를 크게 극복하게 됩니다. 실제로 Spark의 성능은 Hadoop의 10배에 달하는 속도 우위를 보입니다.

그 외에도 정말 다양한 스파크의 강점이 있습니다. 오늘은 스파크의 장점을 전부 세밀하게 분석해 보거나, 실제로 스파크 엔진을 구동해 보거나 하지는 않을 것입니다. 하둡에 대비한 관점에서 스파크에 대한 특징을 잘 정리한 아래 글을 참고해 주시기 바랍니다. 그러나 주의할 점은, 그렇다고 스파크가 하둡에 절대 우위를 보인다는 것은 아닙니다. 하둡은 스파크가 제공하지 않는 HDFS같은 분산 파일 시스템을 제공하고 있습니다. 스파크 자체가 하둡 기반으로 구동하는 것을 목적으로 만들어졌기 때문에, 꼭 하둡이 필요한 것은 아니지만 하둡 기반으로 돌아가는 것을 추천합니다.

- [ITWorld:"하둡을 제압한 빅데이터 플랫폼" 아파치 스파크란 무엇인가](http://www.itworld.co.kr/insight/147556)

하지만 오늘 간과할 수 없는 스파크의 특징 중 하나는, 자바(Java), 스칼라(Scala), 파이썬(Python), R 프로그래밍 언어를 위한 네이티브 바인딩을 제공하고 SQL, 스트리밍 데이터, 머신러닝, 그래프 프로세싱을 지원한다는 점입니다. 스파크를 사용하게 되면, 마치 내 컴퓨터에 있는 csv파일을 Pandas로 읽어 DataFrame으로 분석하는 것처럼, 내 컴퓨터에는 저장도 하기 힘들만큼의 데이터를 DataFrame으로 처리할 수 있다는 장점이 있습니다. 사용자 입장에서는 스파크 뒤에 있는 수많은 컴퓨터들에 분산저장되어 있는 거대한 데이터가 전부 내가 쓰고 있는 컴퓨터 한 대에 들어있는 것처럼 착각하게 만드는 클러스터 컴퓨터이 있습니다. 그리고, 파이썬을 통해 다룰 수 있다는 것이 무엇보다도 두드러진 장점이 아닐까요?

## 함수, 파이썬의 1급 시민

파이썬을 이용한 맵리듀스 프로그래밍에 앞서 언급해야 할 중요한 내용이 있습니다.

바로 파이썬에서 함수가 *1급 시민*이라는 점입니다. 이 특징은 함수형 프로그래밍 개념으로 파이썬에서 맵리듀스를 활용하는 것을 매우 강력하게 합니다.

1급 시민이라니.. 뭔가 특별한 대우를 받을 것만 같죠? 네 맞습니다. 파이썬에서는 함수가 매우 특별합니다. 그리고 함수가 1급 시민이기 때문에 파이썬에서는 함수형 프로그래밍이 가능한 것입니다.

"1급 시민"이란 말은 함수가 다음과 같은 기능을 할 수 있음을 의미합니다.

- 함수는 다른 함수의 인자로 전달될 수 있다. (= 인자(매개변수)로 전달이 가능)
- 함수는 변수에 할당될 수 있다.
- 함수는 다른 함수의 결과로서 반환될 수 있다. (=다른 함수의 반환값)

파이썬에서 "*모든 것은 객체다*"라는 말을 들어보셨죠? 그중에서도 으뜸은 함수입니다.

함수형 프로그래밍은 모든 것을 객체로 표현하게끔 하는 것입니다. 모순적인 말 같지만, 함수 또한 객체기 때문에 함수를 값으로 할당하고 다른 함수의 결과값으로 반환할 수 있습니다.

파이썬과 같이 함수가 일급 시민인 언어는 JavaScript, Scala, Go가 있고 일급 시민이 아닌 언어는 C, Java 등이 있습니다.

## 함수형 프로그래밍과 맵리듀스

### 데이터 컬렉션

파이썬 자료 구조중에 List, Tuple, Dictionary, Set과 같은 형태의 자료형을 **Collection**이라고 합니다. 다른 말로는 컨테이너라고 합니다.

In [1]:
import collections
print("슝=3")

슝=3


파이썬은 `collections` 컬렉션 모듈을 통해서 위 자료형 외 데이터 컬렉션 타입을 제공하기도 합니다.

`collections` 모듈에서 제공하는 데이터 컬렉션은 아래와 같습니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_8_tN8XqDU.max-800x600.png)

여기서 `dequeue()`, `namedtuple()`, `defaultdict` 등은 자주 사용하니까 알아두시면 좋을 것 같습니다!


컬렉션을 얘기한 이유는 빅데이터라는 대용량의 자료를 표현할 때 이러한 타입으로 저장하는 경우가 매우 많기 때문입니다. 위에 나온 클래스가 그대로 쓰이지는 않지만, 키와 밸류 등 기본적인 철학을 공유하는 경우가 많으니 그 원리를 이해하시길 바랍니다.

### 맵리듀스 기본 함수 알아보기

파이썬에서 함수를 데이터 컬렉션에 적용하는 패턴에는 세가지가 있습니다.

- map
- filter
- reduce

파이썬에 있는 함수들을 이용하여 분산처리 비슷한 연산을 해봅시다. 

- 1) map 연산을 통해 여러 개의 컴퓨터에 연산을 나누고,
- 2) reduce를 통해 여러 개 컴퓨터에 분산하여 연산한 결과를 합치면

우리가 원하는 결과가 나올 수 있겠죠? 그럼 시작해봅시다!



#### map (매핑)

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_9_TZK8pJF.max-800x600.png)

컬렉션의 모든 요소에 함수를 적용합니다. 함수 $M(x)$를 컬렉션 $C$의 항목에 매핑하는 함수를 수식으로 나타내면 아래와 같습니다.

$${\displaystyle M(x): x\in C}$$

그럼 코드로 한번 확인해봅시다! 문자열이 들어있는 데이터 컬렉션 `list`에 `int` 함수를 적용시켜봅시다. 그리고 람다(lambda) 함수를 적용하는 방법을 추가해봅시다.

In [2]:
mynum = ['1','2','3','4']
mynum_int = list(map(int, mynum))  # mynum의 각 원소에 int 함수를 적용
print(mynum_int)

mynum_square = list(map(lambda x : x*x, mynum_int))  # mynum_int의 각 원소 x에 lambda x : x*x 함수를 적용
print(mynum_square)

[1, 2, 3, 4]
[1, 4, 9, 16]


#### filter(필터링)

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_10_Cd3pYVv.max-800x600.png)

컬랙션 내의 요소를 선택합니다. 컬랙션 $C$의 요소 $x$에 대해서 조건 $F(x)$에 부합하는 $x$만을 필터링하는 수식은 다음과 같습니다.

$${\displaystyle x: x\in C, if F(x)}$$

아래는 주어진 리스트에서 양수인 요소만을 필터링하는 코드입니다.


In [3]:
mynum = range(-5, 5)  
mynum_plus = list(filter(lambda x: x > 0, mynum)) # mynum의 각 원소 x에 대해 lambda x: x > 0 인지 체크하는 필터를 적용
print(mynum_plus)

[1, 2, 3, 4]


### reduce(리듀싱)

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_11_giirxK9.max-800x600.png)

reduce(리듀싱)은 컬랙션을 축약하니다. 가장 일반적인 예로는 요소들을 합하는 시그마 연산이 있습니다. 아래 수식은 컬랙션 $C$의 요소 $x$의 합을 구해서 축약합니다.

$${\displaystyle \sum _{x\in C}x}$$

코드로 한번 확인해봅시다.

In [4]:
from functools import reduce
mynum = [1, 2, 3, 4, 5]
add = reduce((lambda x, y: x + y), mynum)  # reduce는 내부에 관리하는 x 변수에 mynum의 각 원소 y를 차례차례 더하여 x에 반영한다.

print(add)

15


파이썬의 `map()`함수와 `reduce()` 함수, 어떤가요? 맨 처음에 위키피디아에서 찾은 맵리듀스와 비슷하다는 것이 느껴지시나요? `map()`함수, `reduce()`함수는 중요한 개념입니다.

왜 함수형 프로그래밍이 중요한지, 위의 예제들을 보면서 조금씩 납득이 되시나요?

Mapreduce는 map과 reduce만으로 이루어져 있는 보편적인 프로그래밍 모델입니다. 이 모델이 보편성을 유지하려면 map이 적용되는 방식과 reduce가 적용되는 방식이 미리 정해져 있으면 안됩니다. 위와 같이 map과 reduce가 적용되는 방식을 규정한 함수를 인자로 받는 함수형 프로그래밍이 가능하다는 것은 Mapreduce 프로그래밍 모델을 매우 유연하고 강력하게 만들어 주게 됩니다.

## 맵리듀스 예제

그럼 이제 한번 예제를 살펴봅시다.

### 주요 개념

맵리듀스는 4가지 단계가 있습니다. **Split - Map - Shuffle - Reduce**. input과 output은 각각의 키-밸류 쌍으로 존재한다고 했습니다. 각 단계별 입력값과 출력값은 아래와 같습니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_12_KrcBI1b.max-800x600.png)

그리고 이 각각의 단계는 크게 **Map Task**와 **Reduce Task**로 구분할 수 있습니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_13_x1MFO6H.max-800x600.png)

### 튜토리얼 - 알파벳 수 찾기

그럼 알파벳 빈도수를 맵리듀스를 이용해 구현해보도록 하겠습니다. 

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/Untitled_4_f2gVq0p.max-800x600.png)

#### Mapper : 입력 데이터를 나누고 매핑

In [5]:
#input data
text = 'hello python'

def mapper(text):
    split = []
    for i in text:
        split.append((i, 1))
    return split

#### Reducer : 매퍼에서 입력받은 값 중 같은 항목은 합칩니다.

In [6]:
def reducer(split):
    out = {}
    for i in split:
        if i[0] not in out.keys():
            out[i[0]] = 1
        else:
            out[i[0]] += 1
    return out

#### MapReduce : 출력은 아래와 같습니다.

In [7]:
reducer(mapper(text))

{'h': 2, 'e': 1, 'l': 2, 'o': 2, ' ': 1, 'p': 1, 'y': 1, 't': 1, 'n': 1}

매우 쉽죠? 이게 바로 빅데이터의 첫걸음이었습니다!

실제 데이터를 사용하여 대량의 데이터를 가공하게 되면 매퍼가 더 복잡해질 것이고, 컴퓨터를 여러대 연결하거나 컴퓨터 자원을 최적화하여 계산하게 될 것입니다.

그럼 이것으로 오늘의 글을 마치겠습니다. ^ㅅ^

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style> .container{width:90% !important;}</style>"))